In [62]:
CSV_PATH       = r"D:\University of Illinois Chicago\Classes\CS521\data files\consolidated_sarcasm_dataset.csv"
GLOVE_EN_PATH  = r"D:\University of Illinois Chicago\Classes\CS521\cs521\glove.6B.300d.txt"
GLOVE_HI_PATH  = r"D:\University of Illinois Chicago\Classes\CS521\cs521\glove-hi.300d.txt"
MBERT = "bert-base-multilingual-cased"
MBERT_OUT    = "./baseline_models/bert_mbert"
MODEL_DIR      = "./baseline_models"
BATCH_SIZE = 16
EPOCHS     = 4

In [42]:
import os, re, emoji, random, time, joblib, numpy as np, pandas as pd, torch
from ftfy import fix_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from torch import nn
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np
import torch
import re
import emoji
from ftfy import fix_text

from datasets import Dataset as HFDataset
from torch.utils.data import Dataset as TorchDataset, DataLoader


from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, precision_score, recall_score, f1_score

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import re, regex
import html, emoji, contractions
from ftfy import fix_text
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate

In [39]:
DEVANAGARI_TO_ROMAN = True
LATIN_BLOCK= r'\p{Latin}'
DEVANAGARI_BLOCK = r'\p{Devanagari}'
MAX_LEN_LSTM   = 60
EMBED_DIM      = 300

In [4]:
censored_swear_words = {
    # Handles s!@#, s#!t, s**t, etc., variations of "shit"
    r'\b[Ss][^A-Za-z\s]{3,4}\b': 'shit',
    r'[Ss][\!\@\#\$\%\^\&\*]*[i1\|][\!\@\#\$\%\^\&\*]*[t]+': 'shit',
    r'[Ss][#!@$%^&*+\-=\[\]{};:\'",.<>?/\\|_~`]{2,}[Tt]\b': 'shit',


    # Handles f*ck, f@ck, f**k, f!ck, F*#k, F***, etc., variations of "fuck"
    r'f[\!\@\#\$\%\^\&\*]*[u][\!\@\#\$\%\^\&\*]*[c][\!\@\#\$\%\^\&\*]*[k]+': 'fuck',
    r'f[\!\@\#\$\%\^\&\*]*[\*]+[c][k]': 'fuck',                # Handles f*ck
    r'f[\!\@\#\$\%\^\&\*]*[\*]+[k]': 'fuck',                   # Handles f***
    r'f[\!\@\#\$\%\^\&\*]*[\*#\!]{1,}[c]?[kz]?': 'fuck',       # Handles F*#k, F***
    r'f[\!\@\#\$\%\^\&\*]*[\w@#$%^&*]+ing\b': 'fucking',        # Hnadles f#%^ing

    # Handles b*tch, b!tch, b1tch, etc., variations of "bitch"
    r'b[\!\@\#\$\%\^\&\*]*[i1\|][\!\@\#\$\%\^\&\*]*[t][\!\@\#\$\%\^\&\*]*[c][\!\@\#\$\%\^\&\*]*[h]+': 'bitch',

    # Handles "fuck" with various suffixes (e.g., "f***'z" -> "fucks")
    r'f[\!\@\#\$\%\^\&\*]*[u][\!\@\#\$\%\^\&\*]*[c][\!\@\#\$\%\^\&\*]*[k]+[\'’]?[zs]?': 'fucks',

    # Handles muthaf*ckin, motherf***ing, etc., variations of "motherfucking"
    r'm[ou]*th[\!\@\#\$\%\^\&\*]*[a][\!\@\#\$\%\^\&\*]*f[\!\@\#\$\%\^\&\*]*[u][\!\@\#\$\%\^\&\*]*[c][\!\@\#\$\%\^\&\*]*[k][\!\@\#\$\%\^\&\*]*[i][\!\@\#\$\%\^\&\*]*[n][\!\@\#\$\%\^\&\*]*': 'motherfucking',
}

In [5]:
hindi_swear_map = {

    # —— “madarchod” ——
    # Roman + common leet/censor + Devanagari
    r'\b(m[a@]d[ae]?[r]?[\W_]*ch[o0]d[a@]?|म[ाअ]*दर[चच]ोद|मादरचोद)\b': 'madarchod',

    # —— “bhenchod / behenchod” ——
    r'\b(b[h]*e+h[e]*n[\W_]*ch[o0]d[a@]?|भेंचोद|बह[ेे]न[चच]ोद|बहनचोद)\b': 'bhenchod',

    # —— “chutiya / ch**iya” ——
    r'\b(c[h]*u+t+i+y*a+|ch[u*]+t+iy?[ae]+|चूतिया|चुतिया|छूतिया)\b': 'chutiya',

    # —— “chu***” shortened form ——
    r'\b(ch[\W_]*u[\W_]*t[\W_]*\*{2,})\b': 'chut***',

    # —— “gandu / gaandu / gaand” ——
    r'\b(g[a@]a*n+d[u]?[aei]?|गां*डू?|गाण्डू?)\b': 'gandu',

    # —— “gaand” stand‑alone  ——
    r'\b(ga+a+nd+|गांड|गांड|गाण्ड)\b': 'gaand',

    # —— “lauda / laudae / laude” —— 
    r'\b(l[a@]u+[d]+[aei]*|लौड़ा|लंड|लुंड)\b': 'lauda',

    # —— “lund” ——
    r'\b(l[u]n+d|लुंड|लंड)\b': 'lund',

    # —— “randi / r@ndi” ——
    r'\b(r[a@]n+d[iy]+|रंडी|रंड़ी)\b': 'randi',

    # —— “harami / haramzada / haramzadi” ——
    r'\b(h[a@]r+a+m[iy]*|हरामी)\b': 'harami',
    r'\b(h[a@]r+a+m[z$]?a+d[a@]|हरामज़ादा|हरामजादा)\b': 'haramzada',
    r'\b(h[a@]r+a+m[z$]?a+d[iy]|हरामज़ादी|हरामजादी)\b': 'haramzadi',

    # —— “kamine / kamina / kaminey” ——
    r'\b(k[a@]m+i+n[e]?[yie]*|कमीना|कमीने|कमीनी)\b': 'kamine',

    # —— “kutta / kutte / kuttiya” ——
    r'\b(k[u]t+t[a@e]*|कुत्ता|कुत्ते|कुते)\b': 'kutta',
    r'\b(k[u]t+t[i]y?a+|कुतिया|कुत्ती)\b': 'kutiya',

    # —— “saala / saale / sali” ——
    r'\b(s+a+a*l+a+|साला|साले)\b': 'saala',
    r'\b(s+a+l+i+|साली)\b': 'sali',

    # —— “bakchod / bakchodi” ——
    r'\b(b[a@]k+ch[o0]d[iy]*|बकचोद|बकचोदी)\b' : 'bakchod',

    # —— milder insults ——
    r'\b(g[a@]dh[a@]a+|गधा|गधे)\b': 'gadha',

    # —— catch‑all “f**k” in Roman Hindi sentences ——
    r'\b(f[\W_]*u[\W_]*c[\W_]*k+)\b': 'fuck',

    # —— emojis or censor blocks containing Devanāgarī letters and stars
    r'[\u0900-\u097F]\*{2,}[\u0900-\u097F]*': 'censored_hindi_word',
}


In [6]:
ascii_emoticons = {
    ':-)': 'smiley_face',
    ':)': 'smiley_face',
    ':-D': 'grinning_face',
    ':D': 'grinning_face',
    ':-(': 'sad_face',
    ':(': 'sad_face',
    ':-P': 'playful_face',
    ':P': 'playful_face',
    ':-p': 'playful_face',
    ':p': 'playful_face',
    ';-)': 'winking_face',
    ';)': 'winking_face',
    ':-O': 'surprised_face',
    ':O': 'surprised_face',
    ':-o': 'surprised_face',
    ':o': 'surprised_face',
    r':-/': 'skeptical_face',
    r':/': 'skeptical_face',
    r':-\|': 'neutral_face',
    r':\|': 'neutral_face',
    '<3': 'heart',
    '^_^': 'happy_face',
    '-_-': 'expressionless_face',
    'o_O': 'confused_face',
    'O_o': 'confused_face',
    'o_o': 'confused_face',
    '>_<': 'frustrated_face',
    'O.o': 'confused_face',
    'o.O': 'confused_face',
    '0_o': 'confused_face',
    'o_0': 'confused_face',
    'xD': 'laughing_face',
    'XD': 'laughing_face',
}


In [7]:
def replace_emojis(text):
    return emoji.demojize(text) if isinstance(text, str) else text

In [8]:
def normalize_text(text):
    if not isinstance(text, str):
        return text
    try:
        # Attempt to encode to 'latin1' and decode to 'utf-8' to fix encoding issues
        clean_text = text.encode('latin1').decode('utf-8')
    except (UnicodeEncodeError, UnicodeDecodeError):
        # Fallback: ignore encoding errors if they occur
        clean_text = text.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')
    return clean_text

In [9]:
def decode_html_entities(text):
    """Decode HTML entities in text, such as '&amp;' to '&'."""
    if not isinstance(text, str):
        return text
    # Use html.unescape to decode all HTML entities
    clean_text = html.unescape(text)
    return clean_text

In [10]:
def remove_html_tags(text):
    if not isinstance(text, str):
        return text
    clean_text = re.sub(r'</?[^>]+>', '', text)
    return clean_text

In [11]:
def replace_urls(text):
    if not isinstance(text, str):
        return text
    # Regex pattern to match URLs
    url_pattern = r'(http[s]?://\S+|www\.\S+)'
    clean_text = re.sub(url_pattern, ' SomeWebLink ', text)
    return clean_text

In [12]:
def replace_emails(text):
    if not isinstance(text, str):
        return text
    # Pattern to match email addresses
    email_pattern = r'[\w\.-]+@[\w\.-]+\.\w+'
    clean_text = re.sub(email_pattern, ' EMAILADDRESS ', text)
    return clean_text

In [13]:
def replace_censored_swear_words(text):
    if not isinstance(text, str):
        return text
    clean_text = text
    for pattern, replacement in censored_swear_words.items():
        # Do not add word boundaries if pattern already contains them
        if pattern.startswith(r'\b') and pattern.endswith(r'\b'):
            clean_pattern = pattern
        else:
            clean_pattern = r'\b' + pattern + r'\b'
        clean_text = re.sub(clean_pattern, replacement, clean_text, flags=re.IGNORECASE)
    return clean_text

In [14]:
def replace_hindi_swears(text: str) -> str:
    if not isinstance(text, str):
        return text
    out = text
    for pat, repl in hindi_swear_map.items():
        out = regex.sub(pat, repl, out, flags=regex.IGNORECASE)
    return out

In [15]:
def replace_mentions(text):
    """Replace mentions with 'SomeTaggedAccount', including when attached to other words."""
    if not isinstance(text, str):
        return text
    # Pattern to match @ followed by word characters
    # Replace mentions with spaces around to prevent word merging
    mention_pattern = r'@([A-Za-z0-9_]+)'
    clean_text = re.sub(mention_pattern, ' SomeTaggedAccount ', text)
    return clean_text

In [16]:
def replace_at_symbol(text):
    if not isinstance(text, str):
        return text
    # Replace '@' when used as 'at'
    # Conditions:
    # - '@' followed by a space
    # - '@' at the end of a string
    # - '@' followed by a number or time format
    # - '@' followed by a capitalized word (assuming location or time)
    clean_text = re.sub(r'@\s', 'at ', text)
    clean_text = re.sub(r'@$', 'at', clean_text)
    clean_text = re.sub(r'@(?=\d)', 'at ', clean_text)
    clean_text = re.sub(r'@ (?=[A-Z])', 'at ', clean_text)
    return clean_text

In [17]:
def remove_hashtags(text):
    if not isinstance(text, str):
        return text

    # Remove '#' symbol in various contexts
    clean_text = re.sub(r'(^|\s)#(\w*[^\s\w]?)', r'\1\2', text)
    clean_text = re.sub(r'(\w+)#([^\s\w]?)', r'\1\2', clean_text)
    clean_text = re.sub(r'([^\s\w])#', r'\1', clean_text)

    # Remove any remaining standalone '#' symbols
    clean_text = re.sub(r'(?<=\s)#(?=\s|$)|(?<=^)#(?=\s|$)', '', clean_text)

    return clean_text

In [18]:
def expand_contractions(text):
    if not isinstance(text, str):
        return text
    expanded_words = [contractions.fix(word) for word in text.split()]
    clean_text = ' '.join(expanded_words)
    return clean_text

In [19]:
def clean_unicode_punct(text: str) -> str:
    """
    Remove punctuation *except* ? ! . , and keep both Latin & Devanagari letters.
    Any other symbol is dropped.
    """
    if not isinstance(text, str):
        return text
    # keep letters, numbers, spaces and selected punctuation
    pattern = rf'[^\s{LATIN_BLOCK}{DEVANAGARI_BLOCK}0-9\?\!\.,]'
    return regex.sub(pattern, ' ', text)

In [20]:
def transliterate_devanagari(text: str) -> str:
    if not DEVANAGARI_TO_ROMAN or not isinstance(text, str):
        return text
    return transliterate(text, sanscript.DEVANAGARI, sanscript.ITRANS)

In [21]:
def remove_emoji_colons(text):
    """Remove colons around emoji descriptions in the text."""
    if not isinstance(text, str):
        return text
    # Regex pattern to match emoji descriptions with colons
    pattern = r':([a-zA-Z0-9_+-]+):'
    # Replace matches with the emoji description without colons
    clean_text = re.sub(pattern, r'\1', text)
    return clean_text

In [22]:
def replace_ascii_emoticons(text):
    """Replace ASCII emoticons in text with their descriptions."""
    if not isinstance(text, str):
        return text

    clean_text = text
    for emoticon, description in ascii_emoticons.items():
        # Escape the emoticon pattern to handle special characters
        escaped_emoticon = re.escape(emoticon)
        # Build the regex pattern to match emoticons not part of words
        pattern = r'(?<!\w)' + escaped_emoticon + r'(?!\w)'
        # Replace the emoticon with the description, ignoring case
        clean_text = re.sub(pattern, f"{description}", clean_text, flags=re.IGNORECASE)
    return clean_text

In [23]:
def replace_trade_mark(text):
    """Replace 'trade_mark' with a single quote (')."""
    if not isinstance(text, str):
        return text
    # Replace 'trade_mark' with "'"
    clean_text = text.replace("trade_mark", "'")
    return clean_text

In [24]:
def remove_extra_whitespace(text):
    if not isinstance(text, str):
        return text
    clean_text = ' '.join(text.split())
    return clean_text

In [25]:
cleaning_functions = [
    replace_emojis,
    normalize_text,
    decode_html_entities,
    remove_html_tags,
    replace_urls,
    replace_emails,
    replace_censored_swear_words,
    replace_hindi_swears,
    replace_mentions,
    replace_at_symbol,
    remove_hashtags,
    expand_contractions,
    clean_unicode_punct,
    transliterate_devanagari,
    remove_emoji_colons,
    replace_ascii_emoticons,
    replace_trade_mark,
    remove_extra_whitespace
]

In [28]:
def apply_cleaning(text: str) -> str:
    """Apply every cleaning function in the exact order given."""
    for fn in cleaning_functions:
        text = fn(text)
    return text.strip()

In [33]:
df = pd.read_csv(CSV_PATH).dropna(subset=["comment", "label"])
df["clean"] = df["comment"].astype(str).map(apply_cleaning)

label_map = {'non_sarcastic': 0, 'sarcastic': 1}
df["label"] = df["label"].astype(str).str.strip().map(label_map)

train_df, test_df = train_test_split(
    df[["clean", "label"]],
    test_size=0.20,
    stratify=df["label"],
    random_state=42,
)

# ─── metrics helper ────────────────────────────────────────────────────────
def metrics(y_true, y_pred, name=""):
    acc  = accuracy_score(y_true, y_pred)
    p, r, f, _ = precision_recall_fscore_support(y_true, y_pred, average="binary")
    print(f"{name:12}  Acc {acc:.4f} | Prec {p:.4f} | Rec {r:.4f} | F1 {f:.4f}")
    return acc, p, r, f

os.makedirs(MODEL_DIR, exist_ok=True)

In [34]:
print("\n🔧 Training TF-IDF + LinearSVM baseline …")

tfidf = TfidfVectorizer(
    min_df=3,
    max_df=0.95,
    ngram_range=(1, 2),
    sublinear_tf=True,
    token_pattern=r"(?u)\b\w\w+\b",
)

X_train = tfidf.fit_transform(train_df["clean"])
X_test  = tfidf.transform(test_df["clean"])

svm = LinearSVC(C=1.0, class_weight="balanced", random_state=42)
svm.fit(X_train, train_df["label"])

pred_svm = svm.predict(X_test)

metrics(test_df["label"], pred_svm, "TFIDF-SVM")

joblib.dump((tfidf, svm), f"{MODEL_DIR}/tfidf_svm.joblib")


🔧 Training TF-IDF + LinearSVM baseline …
TFIDF-SVM     Acc 0.7149 | Prec 0.7509 | Rec 0.7849 | F1 0.7675


['./baseline_models/tfidf_svm.joblib']

In [44]:
print("\n🔧 Training BiLSTM baseline … (loading embeddings once)")

def load_glove(path):
    """
    Load GloVe or FastText embeddings into a dict.
    If the first line is a header (e.g., '351320 300'), skip it automatically.
    """
    glove = {}
    with open(path, 'r', encoding='utf-8') as f:
        first_line = f.readline()
        # If first line is two integers, assume it's a header (FastText format)
        if len(first_line.split()) == 2 and all(x.isdigit() for x in first_line.split()):
            print(f"• Detected header line in {path}, skipping...")
        else:
            # No header; reset file read pointer
            f.seek(0)
        
        for line in f:
            parts = line.rstrip().split()
            if len(parts) < 10:  # sanity: ignore malformed lines
                continue
            word, vec = parts[0], np.asarray(parts[1:], dtype=np.float32)
            glove[word] = vec
    return glove

glove_en = load_glove(GLOVE_EN_PATH)
glove_hi = load_glove(GLOVE_HI_PATH)

token2idx = {"<pad>": 0, "<unk>": 1}
vectors   = [np.zeros(EMBED_DIM), np.random.randn(EMBED_DIM) * 0.05]

def add_word(w):
    if w not in token2idx:
        token2idx[w] = len(token2idx)
        if w in glove_en: vectors.append(glove_en[w])
        elif w in glove_hi: vectors.append(glove_hi[w])
        else: vectors.append(np.random.randn(EMBED_DIM) * 0.05)

for txt in train_df["clean"]:
    for tok in txt.split():
        add_word(tok)

emb_matrix = torch.tensor(np.stack(vectors), dtype=torch.float)

class HinglishDS(TorchDataset): 
    def __init__(self, texts, labels, maxlen=MAX_LEN_LSTM):
        self.texts, self.labels, self.maxlen = texts, labels, maxlen
    def __len__(self): return len(self.texts)
    def __getitem__(self, idx):
        toks  = self.texts.iloc[idx].split()              
        idxs  = [token2idx.get(t, 1) for t in toks][:self.maxlen]
        pad   = [0] * (self.maxlen - len(idxs))
        label = self.labels.iloc[idx]                     
        return torch.tensor(idxs + pad), torch.tensor(label)


train_dl = DataLoader(HinglishDS(train_df["clean"], train_df["label"]),
                      batch_size=64, shuffle=True)
test_dl  = DataLoader(HinglishDS(test_df["clean"],  test_df["label"]),
                      batch_size=128)

class BiLSTM(nn.Module):
    def __init__(self, emb, hidden=128, num_classes=2):
        super().__init__()
        self.embed = nn.Embedding.from_pretrained(emb, freeze=False, padding_idx=0)
        self.lstm  = nn.LSTM(emb.size(1), hidden, bidirectional=True, batch_first=True)
        self.fc    = nn.Linear(hidden * 2, num_classes)
    def forward(self, x):
        emb = self.embed(x)
        _, (h, _) = self.lstm(emb)
        h = torch.cat([h[0], h[1]], dim=1)
        return self.fc(h)

device = "cuda" if torch.cuda.is_available() else "cpu"
model  = BiLSTM(emb_matrix).to(device)
criterion = nn.CrossEntropyLoss()
optimiser = torch.optim.AdamW(model.parameters(), lr=1e-3)

for epoch in range(1, 7):
    model.train()
    for xb, yb in train_dl:
        xb, yb = xb.to(device), yb.to(device)
        optimiser.zero_grad()
        loss = criterion(model(xb), yb)
        loss.backward()
        optimiser.step()
    print(f"   • Epoch {epoch}/6  last-batch loss {loss.item():.4f}")


🔧 Training BiLSTM baseline … (loading embeddings once)
• Detected header line in D:\University of Illinois Chicago\Classes\CS521\cs521\glove-hi.300d.txt, skipping...
   • Epoch 1/6  last-batch loss 0.6009
   • Epoch 2/6  last-batch loss 0.4546
   • Epoch 3/6  last-batch loss 0.2991
   • Epoch 4/6  last-batch loss 0.2513
   • Epoch 5/6  last-batch loss 0.0932
   • Epoch 6/6  last-batch loss 0.0649


In [45]:
model.eval(); preds, truth = [], []
with torch.no_grad():
    for xb, yb in test_dl:
        preds.extend(torch.argmax(model(xb.to(device)), 1).cpu().tolist())
        truth.extend(yb.tolist())

metrics(truth, preds, "BiLSTM")

torch.save(model.state_dict(),       f"{MODEL_DIR}/bilstm_state.pt")
np.save(f"{MODEL_DIR}/vocab.npy",    np.array(list(token2idx.items()), dtype=object))

print("\n  Done – models saved in ./baseline_models/")

BiLSTM        Acc 0.7149 | Prec 0.8145 | Rec 0.6792 | F1 0.7407

  Done – models saved in ./baseline_models/


In [53]:
df = pd.read_csv(CSV_PATH).dropna(subset=["comment", "label"])
label_map = {"non_sarcastic": 0, "sarcastic": 1}
df["label"] = df["label"].astype(str).str.strip().map(label_map)
df["clean"] = df["comment"].astype(str).map(apply_cleaning)

train_df, val_df = train_test_split(
    df[["clean", "label"]], test_size=0.15, stratify=df["label"], random_state=42
)
train_ds = HFDataset.from_pandas(train_df.reset_index(drop=True))
val_ds   = HFDataset.from_pandas(val_df.reset_index(drop=True))

In [56]:
tokenizer = AutoTokenizer.from_pretrained(MBERT)

def tokenize(batch):
    return tokenizer(batch["clean"], padding="max_length",
                     truncation=True, max_length=128)

train_ds = train_ds.map(tokenize, batched=True, remove_columns=["clean"])
val_ds   = val_ds.map(tokenize,   batched=True, remove_columns=["clean"])
train_ds.set_format("torch")
val_ds.set_format("torch")

Map: 100%|██████████| 332/332 [00:00<00:00, 18420.89 examples/s]


In [57]:
model = AutoModelForSequenceClassification.from_pretrained(
    MBERT, num_labels=2
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc   = accuracy_score(labels, preds)
    p,r,f,_ = precision_recall_fscore_support(labels, preds, average="binary")
    return {"accuracy": acc, "precision": p, "recall": r, "f1": f}

In [63]:
args = TrainingArguments(
    output_dir          = MBERT_OUT,
    evaluation_strategy = "epoch",
    save_strategy       = "epoch",
    load_best_model_at_end = True,
    metric_for_best_model  = "f1",
    learning_rate       = 2e-5,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size  = BATCH_SIZE,
    num_train_epochs    = EPOCHS,
    weight_decay        = 0.01,
    fp16                = torch.cuda.is_available(),   # mixed precision if GPU
    report_to           = "none",
    logging_steps       = 25,
)

trainer = Trainer(
    model           = model,
    args            = args,
    train_dataset   = train_ds,
    eval_dataset    = val_ds,
    tokenizer       = tokenizer,
    compute_metrics = compute_metrics,
)

c:\Users\Manan\anaconda3\envs\cs521\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\Manan\AppData\Local\Temp\ipykernel_11288\3324455076.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [64]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.567600,0.587437,0.683735,0.813333,0.613065,0.699140
2,0.481200,0.497178,0.756024,0.730469,0.939698,0.821978
3,0.273500,0.498398,0.780120,0.808824,0.829146,0.818859
4,0.217700,0.550245,0.789157,0.789238,0.884422,0.834123


TrainOutput(global_step=472, training_loss=0.40685607013055836, metrics={'train_runtime': 76.2869, 'train_samples_per_second': 98.313, 'train_steps_per_second': 6.187, 'total_flos': 493333228800000.0, 'train_loss': 0.40685607013055836, 'epoch': 4.0})

In [65]:
os.makedirs(MBERT_OUT, exist_ok=True)
trainer.save_model(MBERT_OUT)
tokenizer.save_pretrained(MBERT_OUT)

('./baseline_models/bert_mbert\\tokenizer_config.json',
 './baseline_models/bert_mbert\\special_tokens_map.json',
 './baseline_models/bert_mbert\\vocab.txt',
 './baseline_models/bert_mbert\\added_tokens.json',
 './baseline_models/bert_mbert\\tokenizer.json')